# Fine Tuning para un modelo de clasificación

Explicación paso a paso para crear un modelo Fine-Tuning de un modelo Bert para la clasificación de texto en categorías. Se agrega la opción para crear datos sintéticos.

**Creado por [Camilo Vega](https://www.linkedin.com/in/camilo-vega-169084b1/)**

# Generación datos

### Datos propios

También se puede usar un data set con datos propios. Es cuestión de seguir los siguientes pasos:

1. Abrir una hoja sheet en Drive
2. Crear un columna llamada "text" y otra "label"
3. En la columna "text" poner los ejemplos de textos que se quieran clasificar
4. En la columna "label" poner la respectiva etiqueta que aplique para cada texto.
5. Guardar el archivo en formato .csv. Se recomienda guardarlo bajo el nombre "train" (train.csv) para que el siguiente código aplique a los datos propios.
6. Subir los datos a Colab. La ruta es ir al símbolo de la carpeta, elegir la opción de subir archivo (el símbolo de la hoja con la flecga) y buscar la base de datos en el computador.







In [1]:
import pandas as pd

# Cargar el archivo Excel
df = pd.read_excel('Train.xlsx')

# Crear una lista con los nombres de las columnas de temas
columnas_temas = ['Macroeconomia', 'Microeconomia', 'Econometria', 'Laboral',
                  'Internacional', 'Desarrollo', 'Publica', 'Ambiental',
                  'Salud', 'Financiera', 'Corportamental', 'Sostenibilidad']

# Verificar y reemplazar valores NaN por 0 en las columnas de etiquetas
df[columnas_temas] = df[columnas_temas].fillna(0)

# Asegurarse de que las etiquetas sean valores enteros
df[columnas_temas] = df[columnas_temas].astype(int)

# Crear un nuevo DataFrame con texto y etiquetas
nuevo_df = df[['Abstract'] + columnas_temas].rename(columns={'Abstract': 'text'})

# Guardar el nuevo DataFrame como CSV
nuevo_df.to_csv("train_numerico.csv", index=False)

# Mostrar los primeros registros del nuevo DataFrame
print(nuevo_df.head())


                                                text  Macroeconomia  \
0  OECD countries and their regions are ageing fa...              0   
1  This paper examines the extent to which change...              0   
2  Economic growth depends on human resources and...              0   
3  We present evidence of the minimum wage effect...              0   
4  Using data on applied-for jobs for the univers...              0   

   Microeconomia  Econometria  Laboral  Internacional  Desarrollo  Publica  \
0              0            0        1              0           0        0   
1              0            0        1              0           0        0   
2              0            0        1              0           0        0   
3              0            0        1              0           0        0   
4              0            0        1              0           0        0   

   Ambiental  Salud  Financiera  Corportamental  Sostenibilidad  
0          0      0           0       

# Entrenamiento

In [2]:
# Paso 11: Instalar los paquetes necesarios para el entrenamiento del modelo
!pip install transformers datasets
!pip install transformers[torch]
!pip install accelerate
!pip install evaluate

import pandas as pd
from datasets import Dataset  # Importar Dataset de datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
import numpy as np
import evaluate

# Paso 12: Cargar el conjunto de datos desde el archivo CSV
dataset = pd.read_csv("train_numerico.csv")

# Asumiendo que las columnas son "text" para el texto y las demás columnas para las etiquetas (Macroeconomia, etc.)
# Convertir etiquetas de one-hot encoding a índices únicos (por ejemplo: [0, 1, 0] -> 1)
labels_columns = dataset.columns[1:]  # Excluir la columna "text"
dataset["label"] = dataset[labels_columns].apply(lambda x: np.argmax(x), axis=1)

# Paso 13: Dividir el conjunto de datos en conjuntos de entrenamiento y evaluación
# Eliminar filas con valores nulos en texto o etiquetas
dataset = dataset.dropna(subset=["text", "label"])

# Asegurarse de que la columna "text" sea de tipo string
dataset["text"] = dataset["text"].astype(str)

# Dividir en entrenamiento y evaluación
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    dataset["text"], dataset["label"], test_size=0.2, random_state=42
)

# Convertir listas en diccionarios
train_dataset = {"text": train_texts.tolist(), "labels": train_labels.tolist()}
eval_dataset = {"text": eval_texts.tolist(), "labels": eval_labels.tolist()}

# Verificar que las listas sean homogéneas
assert all(isinstance(x, str) for x in train_dataset["text"]), "Error: valores no texto en train_texts"
assert all(isinstance(x, (int, np.integer)) for x in train_dataset["labels"]), "Error: valores no enteros en train_labels"

# Convertir diccionarios a objetos Dataset
from datasets import Dataset

train_dataset = Dataset.from_dict(train_dataset)
eval_dataset = Dataset.from_dict(eval_dataset)

# Paso 14: Cargar el tokenizador BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Paso 15: Definir la función para tokenizar
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Paso 16: Aplicar tokenización al conjunto de datos
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Paso 17: Ajustar columnas para incluir etiquetas
tokenized_train_dataset = tokenized_train_dataset.with_format("torch")
tokenized_eval_dataset = tokenized_eval_dataset.with_format("torch")

# Paso 18: Cargar el modelo BERT para clasificación de secuencias
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=len(labels_columns)  # Usar el número total de etiquetas
)

# Paso 19: Definir hiperparámetros de entrenamiento
training_args = TrainingArguments(
    output_dir="econ-classifier-multitopic",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="logs",
    logging_steps=10,
    save_strategy="epoch"
)

# Paso 20: Cargar la métrica de precisión
metric = evaluate.load("accuracy")

# Paso 21: Definir la función para calcular métricas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Paso 22: Crear el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Paso 23: Entrenar el modelo
trainer.train()


Map:   0%|          | 0/44 [00:00<?, ? examples/s]

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-10dd25511e11>:93: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.477159,0.000000
2,2.519700,2.504107,0.090909
3,2.519700,2.513384,0.090909


TrainOutput(global_step=18, training_loss=2.4295082092285156, metrics={'train_runtime': 51.0931, 'train_samples_per_second': 2.584, 'train_steps_per_second': 0.352, 'total_flos': 34733777633280.0, 'train_loss': 2.4295082092285156, 'epoch': 3.0})

In [3]:
# Paso 22: Crear el objeto Trainer (continuación)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics,
)  # Crear objeto Trainer

# Paso 24: Evaluar el modelo
eval_results = trainer.evaluate()  # Evaluar el modelo en el conjunto de evaluación
print(eval_results)  # Mostrar los resultados de la evaluación

# Paso 23: Entrenar el modelo
trainer.train()  # Entrenar el modelo

{'eval_loss': 2.5133841037750244, 'eval_model_preparation_time': 0.0129, 'eval_accuracy': 0.09090909090909091, 'eval_runtime': 0.4563, 'eval_samples_per_second': 24.108, 'eval_steps_per_second': 4.383}


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,No log,2.435973,0.012900,0.090909
2,2.169900,2.511901,0.012900,0.090909
3,2.169900,2.512093,0.012900,0.090909


TrainOutput(global_step=18, training_loss=2.1125787099202475, metrics={'train_runtime': 52.9721, 'train_samples_per_second': 2.492, 'train_steps_per_second': 0.34, 'total_flos': 34733777633280.0, 'train_loss': 2.1125787099202475, 'epoch': 3.0})

# Probar el modelo

In [4]:
# Paso 24: Importar el pipeline de transformers
from transformers import pipeline  # Importar pipeline de transformers

# Paso 25: Definir un prompt de ejemplo
prompt = "food is very very good, and perfect service"

# Paso 26: Crear el pipeline de clasificación multi-etiqueta
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer,
                top_k=None, function_to_apply="sigmoid")  # Usar sigmoid para clasificación multi-etiqueta


# Paso 27: Realizar inferencia con el texto de ejemplo
result = pipe(f"<s>[INST] {prompt} [/INST] \n\n")  # Realizar inferencia con texto de ejemplo

# Paso 28: Imprimir el resultado
print(result)  # Imprimir resultado



Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[[{'label': 'LABEL_4', 'score': 0.7104142904281616}, {'label': 'LABEL_6', 'score': 0.6466866731643677}, {'label': 'LABEL_10', 'score': 0.6280766129493713}, {'label': 'LABEL_11', 'score': 0.6123563051223755}, {'label': 'LABEL_0', 'score': 0.5888110399246216}, {'label': 'LABEL_3', 'score': 0.5473036170005798}, {'label': 'LABEL_1', 'score': 0.5170291662216187}, {'label': 'LABEL_8', 'score': 0.5094852447509766}, {'label': 'LABEL_9', 'score': 0.5033411383628845}, {'label': 'LABEL_5', 'score': 0.4558395743370056}, {'label': 'LABEL_2', 'score': 0.44005802273750305}, {'label': 'LABEL_7', 'score': 0.3869825303554535}]]


In [5]:
# Paso 29: Mejorar la respuesta del modelo

# Diccionario para convertir las etiquetas numéricas a etiquetas originales
etiquetas_originales = {
    0: "Macroeconomia",
    1: "Microeconomia",
    2: "Econometria",
    3: "Laboral",
    4: "Internacional",
    5: "Desarrollo",
    6: "Publica",
    7: "Ambiental",
    8: "Salud",
    9: "Financiera",
    10: "Corportamental",
    11: "Sostenibilidad"
}

# Paso 30: Definir la función para convertir predicciones
def convertir_predicciones(resultados, umbral=0.3):
    # Extraer la lista de resultados si está anidada
    if isinstance(resultados, list) and isinstance(resultados[0], list):
        resultados = resultados[0]

    # Convertir etiquetas y filtrar por umbral
    probabilidades_con_etiquetas = {
        etiquetas_originales[int(item['label'].replace('LABEL_', ''))]: item['score']
        for item in resultados
        if item['score'] >= umbral
    }

    # Ordenar por score en orden descendente y obtener las 3 más altas
    etiquetas_predichas = sorted(probabilidades_con_etiquetas.items(), key=lambda x: x[1], reverse=True)[:3]

    return dict(etiquetas_predichas)



In [6]:
# Paso 31: Ejemplo de inferencia y conversión de predicciones
prompt = "Ageing and productivity growth in OECD regions: Combatting the economic impact of ageing through productivity growth?"
pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer, function_to_apply="sigmoid", top_k=None)

# Obtener resultados
result = pipe(prompt)

# Imprimir resultados para depuración
print(f"Resultados del modelo: {result}")

# Convertir las predicciones
etiquetas_predichas = convertir_predicciones(result, umbral=0.3)

# Imprimir predicciones
print(f"Predicciones: {etiquetas_predichas}")



Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Resultados del modelo: [[{'label': 'LABEL_0', 'score': 0.6503864526748657}, {'label': 'LABEL_4', 'score': 0.6382240653038025}, {'label': 'LABEL_3', 'score': 0.6378462910652161}, {'label': 'LABEL_11', 'score': 0.6257410645484924}, {'label': 'LABEL_10', 'score': 0.5695923566818237}, {'label': 'LABEL_6', 'score': 0.5512751936912537}, {'label': 'LABEL_8', 'score': 0.5280040502548218}, {'label': 'LABEL_1', 'score': 0.4999200105667114}, {'label': 'LABEL_5', 'score': 0.4776308238506317}, {'label': 'LABEL_2', 'score': 0.44765931367874146}, {'label': 'LABEL_9', 'score': 0.44083473086357117}, {'label': 'LABEL_7', 'score': 0.42976275086402893}]]
Predicciones: {'Macroeconomia': 0.6503864526748657, 'Internacional': 0.6382240653038025, 'Laboral': 0.6378462910652161}


# Subir el modelo a HuggingFace

In [7]:
# Paso 32: Importar la función notebook_login de huggingface_hub
from huggingface_hub import notebook_login  # Importar notebook_login de huggingface_hub

# Paso 33: Iniciar sesión en Hugging Face Hub
notebook_login()  # Iniciar sesión en Hugging Face Hub



In [8]:
# Paso 34: Publicar el modelo en Hugging Face Hub
trainer.push_to_hub()  # Publicar modelo en Hugging Face Hub

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Esmarguz/econ-classifier-multitopic/commit/e1e1ac451a522f8818a30d6d86cee5e792aec302', commit_message='End of training', commit_description='', oid='e1e1ac451a522f8818a30d6d86cee5e792aec302', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Esmarguz/econ-classifier-multitopic', endpoint='https://huggingface.co', repo_type='model', repo_id='Esmarguz/econ-classifier-multitopic'), pr_revision=None, pr_num=None)